In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import sys
import pickle
import os
import re
import csv
import codecs
import numpy as np
import importlib
import gensim
import random
import psutil
from sklearn.model_selection import train_test_split
import torch.utils.data
import torch.nn.utils.rnn as rnn_utils
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence
import torch.nn.init as init
import gc
import matplotlib.pyplot as plt
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.cuda.set_device(1)
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
args = {
    "optimizer": 'adam',
    "loss_function":'binary_crossentropy',
    "initializer": 'xavier_uniform_',
    "num_epoch":500,
    "batch_size":1,
    "embed_dim":60,
    "num_lstm":60,
    "hidden_dim":240,
    "max_seq_len":230,
    "MAX_NB_WORDS":200000,
    "VALIDATION_SPLIT":0.1,
    "rate_drop_lstm":0.5,
    "rate_drop_dense":0.5,
    "act":'relu',
    "re_weight":True,
    "init_weight_value" :6.0
}

vocb=[]
for i in range(len(content)):
    for j in range(len(content[i])):
        vocb+=content[i][j].split(" ")
vocb=set(vocb)

word_to_idx = {word: i for i, word in enumerate(vocb)}
idx_to_word = {word_to_idx[word]: word for word in word_to_idx}

EMBEDDING_FILE='/media/sysu502/data/duxin/10G训练好的词向量/60维/Word60.model'
STAMP = 'lstm_%d_%d_%.2f_%.2f' % (args['num_lstm'], args['hidden_dim'], args['rate_drop_lstm'], \
                                                args['rate_drop_dense'])

word2vec = gensim.models.deprecated.word2vec.Word2Vec.load(EMBEDDING_FILE)
nb_words = min(args['MAX_NB_WORDS'], len(vocb)) + 1
embedding_matrix = np.zeros((nb_words, args['embed_dim']))

for word, i in word_to_idx.items():
    if word in word2vec.wv.vocab:
        embedding_matrix[i] = word2vec.wv.word_vec(word)
    else:
        continue


class ScheduledOptim(object):
    """A wrapper class for learning rate scheduling
    """

    def __init__(self, optimizer):
        self.optimizer = optimizer
        self.lr = self.optimizer.param_groups[0]['lr']
        self.current_steps = 0

    def step(self):
        "Step by the inner optimizer"
        self.current_steps += 1
        self.optimizer.step()

    def zero_grad(self):
        "Zero out the gradients by the inner optimizer"
        self.optimizer.zero_grad()

    def lr_multi(self, multi):
        for param_group in self.optimizer.param_groups:
            param_group['lr'] *= multi
        self.lr = self.optimizer.param_groups[0]['lr']

    def set_learning_rate(self, lr):
        self.lr = lr
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr

    @property
    def learning_rate(self):
        return self.lr

def generate_mask_2(values, sent_sizes):
    mask_matrix = np.zeros((len(sent_sizes), max(sent_sizes), values.size(2)))
    for i in range(len(sent_sizes)):
        mask_matrix[i][:sent_sizes[i]][:]=1
    if torch.cuda.is_available():
        mask_matrix = torch.Tensor(mask_matrix).cuda()
    else:
        mask_matrix = torch.Tensor(mask_matrix)
    return values*Variable(mask_matrix)

def generate_mask(lsent_sizes, rsent_sizes):
    mask_matrix=np.zeros((len(lsent_sizes),max(lsent_sizes),max(rsent_sizes)))
    for i in range(len(lsent_sizes)):
        mask_matrix[i][:lsent_sizes[i]][:rsent_sizes[i]]=1
    if torch.cuda.is_available():
        mask_matrix = torch.Tensor(mask_matrix).cuda()
    else:
        mask_matrix = torch.Tensor(mask_matrix)
    return Variable(mask_matrix)

class DecAtt(nn.Module):
    def __init__(self, num_units, num_classes, vocab_size, embedding_size,
                 pretrained_emb, word_att,training=True, project_input=True,
                 use_intra_attention=False, distance_biases=10, max_sentence_length=30):
        super(DecAtt, self).__init__()
        self.vocab_size=vocab_size
        self.num_units = num_units
        self.num_classes = num_classes
        self.project_input = project_input
        self.embedding_size=embedding_size
        self.distance_biases=distance_biases
        self.intra_attention=False
        self.max_sentence_length=max_sentence_length
        self.pretrained_emb=pretrained_emb

        self.bias_embedding=nn.Embedding(max_sentence_length,1)
        self.word_embedding=nn.Embedding.from_pretrained(torch.tensor(pretrained_emb, dtype=torch.float))
        self.linear_layer_project = nn.Linear(embedding_size, num_units, bias=False)

        self.linear_layer_wordatt1 = nn.Sequential(nn.Dropout(p=0.2), nn.Linear(num_units, num_units), nn.ReLU(inplace=True))
        self.linear_layer_wordatt2 = nn.Sequential(nn.Dropout(p=0.2), nn.Linear(num_units, num_units), nn.ReLU(inplace=True))
        
        self.linear_layer_attend = nn.Sequential(nn.Dropout(p=0.2), nn.Linear(num_units, num_units), nn.ReLU(inplace=True),
                                                 nn.Dropout(p=0.2), nn.Linear(num_units, num_units), nn.ReLU(inplace=True))

        self.linear_layer_compare = nn.Sequential(nn.Dropout(p=0.2), nn.Linear(num_units*2, num_units), nn.ReLU(inplace=True),
                                                  nn.Dropout(p=0.2), nn.Linear(num_units, num_units), nn.ReLU(inplace=True))

        self.linear_layer_aggregate = nn.Sequential(nn.Dropout(p=0.2), nn.Linear(num_units*4, num_units), nn.ReLU(inplace=True),
                                                    nn.Dropout(p=0.2), nn.Linear(num_units, num_units), nn.ReLU(inplace=True),
                                                    nn.Linear(num_units, num_classes))
        
        nn.MaxPool2d
        self.init_weight()

    def init_weight(self):
        self.linear_layer_project.weight.data.normal_(0, 0.01)
        self.linear_layer_wordatt1[1].weight.data.normal_(0, 0.01)
        self.linear_layer_wordatt1[1].bias.data.fill_(0)
        self.linear_layer_wordatt2[1].weight.data.normal_(0, 0.01)
        self.linear_layer_wordatt2[1].bias.data.fill_(0)
        self.linear_layer_attend[1].weight.data.normal_(0, 0.01)
        self.linear_layer_attend[1].bias.data.fill_(0)
        self.linear_layer_attend[4].weight.data.normal_(0, 0.01)
        self.linear_layer_attend[4].bias.data.fill_(0)
        self.linear_layer_compare[1].weight.data.normal_(0, 0.01)
        self.linear_layer_compare[1].bias.data.fill_(0)
        self.linear_layer_compare[4].weight.data.normal_(0, 0.01)
        self.linear_layer_compare[4].bias.data.fill_(0)
        self.linear_layer_aggregate[1].weight.data.normal_(0, 0.01)
        self.linear_layer_aggregate[1].bias.data.fill_(0)
        self.linear_layer_aggregate[4].weight.data.normal_(0, 0.01)
        self.linear_layer_aggregate[4].bias.data.fill_(0)

    def attention_softmax3d(self,raw_attentions):
        reshaped_attentions = raw_attentions.view(-1, raw_attentions.size(2))
        out=nn.functional.softmax(reshaped_attentions, dim=1)
        return out.view(raw_attentions.size(0),raw_attentions.size(1),raw_attentions.size(2))
    
    def attention_softmax2d(self,raw_attentions):
        reshaped_attentions = raw_attentions
        out=nn.functional.softmax(reshaped_attentions, dim=0)
        return out

    def _transformation_input(self,embed_sent):
        embed_sent=self.word_embedding(embed_sent)
        embed_sent=self.linear_layer_project(embed_sent)
        result=embed_sent
        if self.intra_attention:
            f_intra = self.linear_layer_intra(embed_sent)
            f_intra_t = torch.transpose(f_intra, 1, 2)
            raw_attentions = torch.matmul(f_intra, f_intra_t)
            time_steps=embed_sent.size(1)
            r = torch.arange(0, time_steps)
            #从start到end的一组序列值，以1为间隔，不包括两端
            r_matrix=r.view(1,-1).expand(time_steps,time_steps)
            raw_index=r_matrix-r.view(-1,1)
            clipped_index=torch.clamp(raw_index,0,self.distance_biases-1)
            clipped_index=Variable(clipped_index.long())
            if torch.cuda.is_available():
                clipped_index=clipped_index.cuda()
            bias=self.bias_embedding(clipped_index)
            bias=torch.squeeze(bias)
            raw_attentions += bias
            attentions = self.attention_softmax3d(raw_attentions)
            attended = torch.matmul(attentions, embed_sent)
            result =torch.cat([embed_sent,attended],2)
        return result

    def attend(self,sent1,sent2):
        repr1=self.linear_layer_attend(sent1)
        repr2=self.linear_layer_attend(sent2)
        repr2=torch.transpose(repr2,1,2)
        raw_attentions = torch.matmul(repr1, repr2)

        att_sent1 = self.attention_softmax3d(raw_attentions)
        beta=torch.matmul(att_sent1,sent2) #input2_soft

        raw_attentions_t=torch.transpose(raw_attentions,1,2).contiguous()
        att_sent2 = self.attention_softmax3d(raw_attentions_t)
        alpha=torch.matmul(att_sent2,sent1) #input1_soft

        return alpha, beta

    def compare(self,sentence,soft_alignment):
        sent_alignment=torch.cat([sentence, soft_alignment],2)
        out = self.linear_layer_compare(sent_alignment)
        return out

    def aggregate(self,v1, v2):
        v1_sum=torch.sum(v1,1)
        v2_sum=torch.sum(v2,1)
        return torch.cat([v1_sum,v2_sum],1)

    def forward(self,train_1,train_2,train1_att,train2_att):
        maxp_dvector=[]
#         len1=0
#         if len(train_1)<len(train_2):
#             len1=len(train_1)
#         else:
#             len1=len(train_2)
        for i in range(len(train_2)):
            j1=len(train_1[0])-1
            while 1:
                if train_1[0,j1]==0:
                    j1-=1
                else:
                    break

            j2=len(train_2[i])-1
            while 1:
                if train_2[i,j2]==0:
                    j2-=1
                else:
                    break
        
            sent1=self._transformation_input(train_1[0,:j1+1].unsqueeze(0))
            sent2=self._transformation_input(train_2[i,:j2+1].unsqueeze(0))
            alpha, beta = self.attend(sent1, sent2)
            v1=self.compare(sent1,beta)
            v2=self.compare(sent2,alpha)
            logits=self.aggregate(v1,v2)
            if i==0:
                maxp_dvector=logits
            else:
                maxp_dvector=torch.cat([maxp_dvector,logits],0)
#         maxp_dvector=maxp_dvector.unsqueeze(0)
        maxp_dvector=torch.mean(maxp_dvector,0)
        sent1=self._transformation_input(train1_att.unsqueeze(0))
        sent2=self._transformation_input(train2_att.unsqueeze(0))
        alpha, beta = self.attend(sent1, sent2)
        v1=self.compare(sent1,beta)
        v2=self.compare(sent2,alpha)
        uvector=self.aggregate(v1,v2)
        concat_vector=torch.cat([maxp_dvector.unsqueeze(0),uvector],1)
        logits=self.linear_layer_aggregate(concat_vector)
            
        return logits

train_1=[]
train_2=[]
test_1=[]
test_2=[]
train1_att=[]
test1_att=[]
train2_att=[]
test2_att=[]
maxl=0
for i in range(len(train1)):
    train_1.append([])
    for k in range(len(train1[i])):
        x=train1[i][k].split(" ")
        if maxl<len(x):
            maxl=len(x)
        x1= [word_to_idx[j] for j in x]
        train_1[-1].append(x1)
for i in range(len(train2)):
    train_2.append([])
    for k in range(len(train2[i])):
        x=train2[i][k].split(" ")
        if maxl<len(x):
            maxl=len(x)
        x1= [word_to_idx[j] for j in x]
        train_2[-1].append(x1)
for i in range(len(test1)):
    test_1.append([])
    for k in range(len(test1[i])):
        x=test1[i][k].split(" ")
        if maxl<len(x):
            maxl=len(x)
        x1= [word_to_idx[j] for j in x]
        test_1[-1].append(x1)
for i in range(len(test2)):
    test_2.append([])
    for k in range(len(test2[i])):
        x=test2[i][k].split(" ")
        if maxl<len(x):
            maxl=len(x)
        x1= [word_to_idx[j] for j in x]
        test_2[-1].append(x1)

for i in range(len(wordstoprank1001)):
    train1_att.append([])
    for j in range(len(wordstoprank1001[i])):
        x= [word_to_idx[k] for k in wordstoprank1001[i][j]]
        train1_att[-1].append(x)
for i in range(len(wordstoptrank1001)):
    test1_att.append([])
    for j in range(len(wordstoptrank1001[i])):
        x= [word_to_idx[k] for k in wordstoptrank1001[i][j]]
        test1_att[-1].append(x)
for i in range(len(wordstoprank1002)):
    train2_att.append([])
    for j in range(len(wordstoprank1002[i])):
        x= [word_to_idx[k] for k in wordstoprank1002[i][j]]
        train2_att[-1].append(x)
for i in range(len(wordstoptrank1002)):
    test2_att.append([])
    for j in range(len(wordstoptrank1002[i])):
        x= [word_to_idx[k] for k in wordstoptrank1002[i][j]]
        test2_att[-1].append(x)
    
args['max_seq_len']=maxl

print(maxl)

for i in range(len(train_1)):
    for j in range(len(train_1[i])):
        while 1:
            if len(train_1[i][j])<args['max_seq_len']:
                train_1[i][j].append(0)
            else:
                break
    for j in range(len(train_2[i])):
        while 1:
            if len(train_2[i][j])<args['max_seq_len']:
                train_2[i][j].append(0)
            else:
                break

for i in range(len(test_1)):
    for j in range(len(test_1[i])):
        while 1:
            if len(test_1[i][j])<args['max_seq_len']:
                test_1[i][j].append(0)
            else:
                break
    for j in range(len(test_2[i])):
        while 1:
            if len(test_2[i][j])<args['max_seq_len']:
                test_2[i][j].append(0)
            else:
                break

# train_1=Variable(torch.LongTensor(train_1))
# train_2=Variable(torch.LongTensor(train_2))
# test_1=Variable(torch.LongTensor(test_1))
# test_2=Variable(torch.LongTensor(test_2))
# train1_att=Variable(torch.LongTensor(train1_att))
# train2_att=Variable(torch.LongTensor(train2_att))
# test1_att=Variable(torch.LongTensor(test1_att))
# test2_att=Variable(torch.LongTensor(test2_att))
# if torch.cuda.is_available():
#     train_1 = train_1.cuda()
#     train_2 = train_2.cuda()
#     test_1=test_1.cuda()
#     test_2=test_2.cuda()
#     train1_att=train1_att.cuda()
#     train2_att=train2_att.cuda()
#     test1_att=test1_att.cuda()
#     test2_att=test2_att.cuda()

model = DecAtt(200,2,nb_words,args["embed_dim"],embedding_matrix,100)
# num_units, num_classes, vocab_size, embedding_size,pretrained_emb
if torch.cuda.is_available():
    model = model.cuda()
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad,model.parameters()),lr=0.0005)
# optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad,model.parameters()), lr = 0.0005,momentum=0.9,nesterov=True)
optimizer=torch.optim.Adagrad(filter(lambda p: p.requires_grad,model.parameters()), lr=0.0012, weight_decay=0)
optimizer = ScheduledOptim(optimizer)
lr_mult = (1 / 1e-5) ** (1 / 100)
lr = []
losses = []
best_loss = 1e9
pmacc=0
macc=0
decaylr=0.7

for epoch in range(args['num_epoch']):
    model.train()
    print('*' * 10)
    print('epoch {}'.format(epoch + 1))
    running_loss = 0
    yp=0
    yn=0
    tp=0
    tn=0
    for i in range(len(train_1)):
        
        len1=0
        if len(train_1[i])<len(train_2[i]):
            len1=len(train_1[i])
        else:
            len1=len(train_2[i])
        
        if len1/text_group<1:
            continue
        for j in range(math.floor(len1/text_group)):
            out=model(Variable(torch.LongTensor(train_1[i][j*text_group:(j+1)*text_group])).cuda(),Variable(torch.LongTensor(train_2[i][j*text_group:(j+1)*text_group])).cuda(),Variable(torch.LongTensor(train1_att[i][j])).cuda(),Variable(torch.LongTensor(train2_att[i][j])).cuda())
        
            loss = criterion(out, torch.LongTensor([1]).cuda())
            running_loss += loss.data.item()
            pred_y = torch.max(out.cpu(), 1)[1].data.numpy().squeeze()
            yp+=1
            if pred_y.item()==1:
                tp+=1
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            for i1 in range(1):
                while 1:
                    index1=random.randint(0,len(train_1)-1)
                    if index1!=i and len(train_2[index1])>text_group:
                        break

                index2=math.floor(random.randint(0,len(train_2[index1])-text_group-1)/text_group)
                out=model(Variable(torch.LongTensor(train_1[i][j*text_group:(j+1)*text_group])).cuda(),Variable(torch.LongTensor(train_2[index1][index2*text_group:(index2+1)*text_group])).cuda(),Variable(torch.LongTensor(train1_att[i][j])).cuda(),Variable(torch.LongTensor(train2_att[index1][index2])).cuda())

                loss = criterion(out, torch.LongTensor([0]).cuda())
                running_loss += loss.data.item()
                pred_y = torch.max(out.cpu(), 1)[1].data.numpy().squeeze()
                yn+=1
                if pred_y.item()==0:
                    tn+=1

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

    torch.cuda.empty_cache()
    
    print("train_end")
    print('trainn acc:', tn, yn, tn/yn)
    print('trainp acc:', tp, yp, tp/yp)
    print(psutil.Process(os.getpid()).memory_info().rss)
#     if (tn/yn+tp/yp)/2-decaylr>0:
#         decaylr+=0.01
#         optimizer.set_learning_rate(optimizer.learning_rate/lr_mult)
    model.eval()
    yp=0
    yn=0
    tp=0
    tn=0
    pre_i=0
    running_loss1 = 0
    for i in range(len(test_1)):
        len1=0
        if len(test_1[i])<len(test_2[i]):
            len1=len(test_1[i])
        else:
            len1=len(test_2[i])
            
        if len1/text_group<1:
            continue
        for j in range(math.floor(len1/text_group)):
            out=model(Variable(torch.LongTensor(test_1[i][j*text_group:(j+1)*text_group])).cuda(),Variable(torch.LongTensor(test_2[i][j*text_group:(j+1)*text_group])).cuda(),Variable(torch.LongTensor(test1_att[i][j])).cuda(),Variable(torch.LongTensor(test2_att[i][j])).cuda())

            loss = criterion(out, torch.LongTensor([1]).cuda())
            running_loss += loss.data.item()
            pred_y = torch.max(out.cpu(), 1)[1].data.numpy().squeeze()
            yp+=1
            if pred_y.item()==1:
                tp+=1
            optimizer.zero_grad()

            for i1 in range(1):
                while 1:
                    index1=random.randint(0,len(test_1)-1)
                    if index1!=i and len(test_2[index1])>text_group:
                        break

                index2=math.floor(random.randint(0,len(test_2[index1])-text_group-1)/text_group)
                out=model(Variable(torch.LongTensor(test_1[i][j*text_group:(j+1)*text_group])).cuda(),Variable(torch.LongTensor(test_2[index1][index2*text_group:(index2+1)*text_group])).cuda(),Variable(torch.LongTensor(test1_att[i][j])).cuda(),Variable(torch.LongTensor(test2_att[index1][index2])).cuda())

                loss = criterion(out, torch.LongTensor([0]).cuda())
                running_loss1 += loss.data.item()
                pred_y = torch.max(out.cpu(), 1)[1].data.numpy().squeeze()
                yn+=1
                if pred_y.item()==0:
                    tn+=1
                optimizer.zero_grad()
    print("test_end")
    print('testn acc:', tn, yn, tn/yn)
    print('testp acc:', tp, yp, tp/yp)
    print(psutil.Process(os.getpid()).memory_info().rss)
    
    
    print('Loss: {}'.format(running_loss / len(train_1)))
    print('Loss: {}'.format(running_loss1 / len(test_1)))

314
**********
epoch 1
train_end
trainn acc: 4102 6621 0.6195438755475003
trainp acc: 5735 6621 0.8661833559885214
6049472512
test_end
testn acc: 509 780 0.6525641025641026
testp acc: 649 780 0.8320512820512821
6049472512
Loss: 8.232313479488319
Loss: 3.8520520508289335
**********
epoch 2
train_end
trainn acc: 4312 6621 0.6512611388007854
trainp acc: 5867 6621 0.8861199214620148
6049472512
test_end
testn acc: 487 780 0.6243589743589744
testp acc: 675 780 0.8653846153846154
6049472512
Loss: 7.6451254472153805
Loss: 4.084273035526276
**********
epoch 3
train_end
trainn acc: 4429 6621 0.6689321854704727
trainp acc: 5886 6621 0.8889895786135025
6049472512
test_end
testn acc: 495 780 0.6346153846153846
testp acc: 666 780 0.8538461538461538
6049472512
Loss: 7.39048613761914
Loss: 4.00371974080801
**********
epoch 4
train_end
trainn acc: 4462 6621 0.6739163268388461
trainp acc: 5909 6621 0.8924633741126718
6049472512
test_end
testn acc: 508 780 0.6512820512820513
testp acc: 675 780 0.86538461

train_end
trainn acc: 5440 6621 0.821628152847002
trainp acc: 6083 6621 0.9187433922368222
6049472512
test_end
testn acc: 633 780 0.8115384615384615
testp acc: 702 780 0.9
6049472512
Loss: 4.833106032456947
Loss: 2.6298003578186036
**********
epoch 33
train_end
trainn acc: 5457 6621 0.8241957408246489
trainp acc: 6095 6621 0.9205558072798671
6049472512
test_end
testn acc: 640 780 0.8205128205128205
testp acc: 700 780 0.8974358974358975
6049472512
Loss: 4.811715563519075
Loss: 2.594316924214363
**********
epoch 34
train_end
trainn acc: 5466 6621 0.8255550521069325
trainp acc: 6131 6621 0.9259930524090016
6051598336
test_end
testn acc: 619 780 0.7935897435897435
testp acc: 712 780 0.9128205128205128
6052110336
Loss: 4.733756887718914
Loss: 3.092359281182289
**********
epoch 35
train_end
trainn acc: 5483 6621 0.8281226400845794
trainp acc: 6126 6621 0.9252378794743996
6062596096
test_end
testn acc: 652 780 0.8358974358974359
testp acc: 708 780 0.9076923076923077
6063120384
Loss: 4.7168327

train_end
trainn acc: 5627 6621 0.8498716206011176
trainp acc: 6184 6621 0.9339978855157831
6367485952
test_end
testn acc: 632 780 0.8102564102564103
testp acc: 704 780 0.9025641025641026
6368010240
Loss: 4.152944504210896
Loss: 2.663821920156479
**********
epoch 64
train_end
trainn acc: 5634 6621 0.8509288627095605
trainp acc: 6199 6621 0.9362634043195892
6378496000
test_end
testn acc: 646 780 0.8282051282051283
testp acc: 718 780 0.9205128205128205
6379020288
Loss: 4.180696082897441
Loss: 2.633631422519684
**********
epoch 65
train_end
trainn acc: 5637 6621 0.8513819664703217
trainp acc: 6209 6621 0.9377737501887933
6389248000
test_end
testn acc: 634 780 0.8128205128205128
testp acc: 707 780 0.9064102564102564
6389768192
Loss: 4.174576439404737
Loss: 2.8642338716983797
**********
epoch 66
train_end
trainn acc: 5638 6621 0.8515330010572421
trainp acc: 6203 6621 0.9368675426672708
6400253952
test_end
testn acc: 647 780 0.8294871794871795
testp acc: 711 780 0.9115384615384615
6400778240

In [39]:
train_11=Variable(torch.LongTensor(train_1[i][j*10:(j+1)*10])).cuda()
train_22=Variable(torch.LongTensor(train_2[i][j*10:(j+1)*10])).cuda()
train1_att1=Variable(torch.LongTensor(train1_att[i][j*10:(j+1)*10])).cuda()
train2_att1=Variable(torch.LongTensor(train2_att[i][j*10:(j+1)*10])).cuda()

maxp_dvector=[]
for i1 in range(len(train_22)):
    j1=len(train_11[0])-1
    while 1:
        if train_11[0,j1]==0:
            j1-=1
        else:
            break

    j2=len(train_22[i1])-1
    while 1:
        if train_22[i1,j2]==0:
            j2-=1
        else:
            break

    sent1=model._transformation_input(train_11[0,:j1+1].unsqueeze(0))
    sent2=model._transformation_input(train_22[i1,:j2+1].unsqueeze(0))
    alpha, beta = model.attend(sent1, sent2)
    v1=model.compare(sent1,beta)
    v2=model.compare(sent2,alpha)
    logits=model.aggregate(v1,v2)
    if i1==0:
        maxp_dvector=logits
    else:
        maxp_dvector=torch.cat([maxp_dvector,logits],0)
#         maxp_dvector=maxp_dvector.unsqueeze(0)
maxp_dvector=torch.max(maxp_dvector,0)[0]
sent1=model._transformation_input(train1_att1.unsqueeze(0))
sent2=model._transformation_input(train2_att1.unsqueeze(0))
alpha, beta = model.attend(sent1, sent2)
v1=model.compare(sent1,beta)
v2=model.compare(sent2,alpha)
uvector=model.aggregate(v1,v2)
concat_vector=torch.cat([maxp_dvector.unsqueeze(0),uvector],1)
logits=model.linear_layer_aggregate(concat_vector)

In [71]:
input1 = open('/home/sysu502/Public/duxin/text4/douban_broadcaste44.txt', 'r',encoding="utf-8")
content=eval(input1.read())
input1.close()

In [15]:
wordstoptrank1001=[]
wordstoptrank1002=[]
for i in range(len(test1)):
    wordstoptrank1001.append([])
    for j in range(math.floor(len(test1[i])/text_group)):
        wordstoptrank1001[-1].append(textrank.textrank(test1[i][j*text_group:(j+1)*text_group],100))
for i in range(len(test2)):
    wordstoptrank1002.append([])
    for j in range(math.floor(len(test2[i])/text_group)):
        wordstoptrank1002[-1].append(textrank.textrank(test2[i][j*text_group:(j+1)*text_group],100))
        
test1_att=[]
test2_att=[]
for i in range(len(wordstoptrank1001)):
    test1_att.append([])
    for j in range(len(wordstoptrank1001[i])):
        x= [word_to_idx[k] for k in wordstoptrank1001[i][j]]
        test1_att[-1].append(x)
for i in range(len(wordstoptrank1002)):
    test2_att.append([])
    for j in range(len(wordstoptrank1002[i])):
        x= [word_to_idx[k] for k in wordstoptrank1002[i][j]]
        test2_att[-1].append(x)

ra=[]
ra1=[]
for i in range(len(test_1)):
    ra.append([])
    ra1.append([])
    for j in range(len(test_2)):
        test_output=model(Variable(torch.LongTensor(test_1[i][0:text_group])).cuda(),Variable(torch.LongTensor(test_2[j][0:text_group])).cuda(),Variable(torch.LongTensor(test1_att[i][0])).cuda(),Variable(torch.LongTensor(test2_att[j][0])).cuda())

        count1=0
        count2=0
        pred_y = torch.max(test_output.cpu(), 1)[1].data.numpy().squeeze()
        if pred_y==1:
            count2=1
        else:
            count2=0
        for i2 in range(len(test_output.cpu())):
            count1+=F.softmax(test_output.cpu()[i2])[1].data.numpy().squeeze()
        ra[-1].append(count1)
        ra1[-1].append(count2)
        optimizer.zero_grad()

    print(i)
    print(max(ra[-1]),max(ra1[-1]))
    print(ra[-1][i],ra1[-1][i])

/home/sysu502/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0
0.9780640006065369 1
0.9780640006065369 1
1
0.9747328162193298 1
0.9747328162193298 1
2
0.9214175939559937 1
0.8920028805732727 1
3
0.966833770275116 1
0.966833770275116 1
4
0.9799709916114807 1
0.9799709916114807 1
5
0.9999985694885254 1
0.9999985694885254 1
6
0.9697672128677368 1
0.9293823838233948 1
7
0.9999342560768127 1
0.9999342560768127 1
8
0.9707826375961304 1
0.9422178268432617 1
9
0.9568617343902588 1
0.9055628776550293 1
10
0.9990440011024475 1
0.9990440011024475 1
11
0.9628667235374451 1
0.8976401090621948 1
12
0.9796913266181946 1
0.9796913266181946 1
13
0.9425537586212158 1
0.8762763142585754 1
14
0.9806063771247864 1
0.7017678618431091 1
15
0.9528141617774963 1
0.9528141617774963 1
16
0.9743732810020447 1
0.9479549527168274 1
17
0.922212541103363 1
0.922212541103363 1
18
0.9937052130699158 1
0.9937052130699158 1
19
0.9485782384872437 1
0.4425576329231262 0
20
0.950064480304718 1
0.6483310461044312 1
21
0.920221745967865 1
0.8780270218849182 1
22
0.9606241583824158 1
0.

In [7]:
model.eval()
wordstoptrank1001=[]
wordstoptrank1002=[]
for i in range(len(test1)):
    wordstoptrank1001.append(textrank.textrank(test1[i],100))
for i in range(len(test2)):
    wordstoptrank1002.append(textrank.textrank(test2[i],100))
test1_att=[]
test2_att=[]

for i in range(len(wordstoptrank1001)):
    x= [word_to_idx[k] for k in wordstoptrank1001[i]]
    test1_att.append(x)
for i in range(len(wordstoptrank1002)):
    x= [word_to_idx[k] for k in wordstoptrank1002[i]]
    test2_att.append(x)
    
ra=[]
ra1=[]
for i in range(len(test_1)):
    ra.append([])
    ra1.append([])
    for j in range(len(test_2)):
        test_output=model(Variable(torch.LongTensor(test_1[i])).cuda(),Variable(torch.LongTensor(test_2[j])).cuda(),Variable(torch.LongTensor(test1_att[i])).cuda(),Variable(torch.LongTensor(test2_att[j])).cuda())

        count1=0
        count2=0
        pred_y = torch.max(test_output.cpu(), 1)[1].data.numpy().squeeze()
        if pred_y==1:
            count2=1
        else:
            count2=0
        for i2 in range(len(test_output.cpu())):
            count1+=F.softmax(test_output.cpu()[i2])[1].data.numpy().squeeze()
        ra[-1].append(count1)
        ra1[-1].append(count2)
        optimizer.zero_grad()

    print(i)
    print(max(ra[-1]),max(ra1[-1]))
    print(ra[-1][i],ra1[-1][i])

/home/sysu502/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0
0.9999608993530273 1
0.9999608993530273 1
1
1.0 1
0.9999949932098389 1
2
0.9999999403953552 1
0.9059488773345947 1
3
0.9999914765357971 1
0.9999695420265198 1
4
0.9999656081199646 1
0.9999656081199646 1
5
1.0 1
1.0 1
6
0.9999998211860657 1
0.9999517202377319 1
7
1.0 1
1.0 1
8
1.0 1
1.0 1
9
0.999998152256012 1
0.1848207414150238 0
10
0.9999995231628418 1
0.9999995231628418 1
11
0.9999465346336365 1
0.999932050704956 1
12
0.9999904632568359 1
0.983428418636322 1
13
0.9999997019767761 1
0.9609155654907227 1
14
0.9834640622138977 1
0.9531801342964172 1
15
0.999940812587738 1
0.3934744894504547 0
16
0.9999996423721313 1
0.999896764755249 1
17
0.9999887347221375 1
0.9933969974517822 1
18
0.9999999403953552 1
0.12020143866539001 0
19
0.9999956488609314 1
0.9992242455482483 1
20
0.9989805221557617 1
0.38686880469322205 0
21
0.9995787143707275 1
0.9995787143707275 1
22
1.0 1
0.9110967516899109 1
23
0.9999997019767761 1
0.9997721314430237 1
24
1.0 1
0.9973542094230652 1
25
0.9999979734420776 1

In [23]:
ra1=np.matrix(ra[0:100][0:100])
rat=ra1.T
ac=0
maac=0
for i in range(len(ra1)):
    r1=np.argmax(ra1[i])
    r2=np.argmax(rat[i])
    if r1==r2:
        maac+=1
    if r1==r2 and r1==i:
        ac+=1
recall=ac/len(ra1)
precise=ac/maac
print("recall:",recall)
print("precise",precise)

recall: 0.41
precise 0.9534883720930233


In [15]:
all1=0
correct1=0
for i in range(len(ra1)):
    for j in range(len(ra1[i])):
        if ra1[i][j]==1:
            all1+=1
            if i==j:
                correct1+=1
print("precise:",correct1/all1)
print("recall",correct1/100)

precise: 0.06795366795366796
recall 0.88


In [1]:
input1 = open('/home/sysu502/Public/duxin/content_new961.txt', 'r',encoding="utf-8")
content=eval(input1.read())
input1.close()

In [2]:
from sklearn.model_selection import train_test_split
y=[]
for i in range(len(content)):
    y.append(0)
x_train, x_test, y_train, y_test = train_test_split(content, y, train_size=861/961)

/home/sysu502/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [3]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from __future__ import absolute_import, unicode_literals
import sys
from operator import itemgetter
from collections import defaultdict

class UndirectWeightedGraph:
    d = 0.85

    def __init__(self):
        self.graph = defaultdict(list)

    def addEdge(self, start, end, weight):
        # use a tuple (start, end, weight) instead of a Edge object
        self.graph[start].append((start, end, weight))
        self.graph[end].append((end, start, weight))

    def rank(self):
        ws = defaultdict(float)
        outSum = defaultdict(float)

        wsdef = 1.0 / (len(self.graph) or 1.0)
        for n, out in self.graph.items():
            ws[n] = wsdef
            outSum[n] = sum((e[2] for e in out), 0.0)

        # this line for build stable iteration
        sorted_keys = sorted(self.graph.keys())
        for x in range(10):  # 10 iters
            for n in sorted_keys:
                s = 0
                for e in self.graph[n]:
                    s += e[2] / outSum[e[1]] * ws[e[1]]
                ws[n] = (1 - self.d) + self.d * s

        (min_rank, max_rank) = (sys.float_info[0], sys.float_info[3])

        for w in ws.values():
            if w < min_rank:
                min_rank = w
            if w > max_rank:
                max_rank = w

        for n, w in ws.items():
            # to unify the weights, don't *100.
            ws[n] = (w - min_rank / 10.0) / (max_rank - min_rank / 10.0)

        return ws

class TextRank:

    def __init__(self):
        self.span = 3

    def pairfilter(self, wp):
        return (wp.flag in self.pos_filt and len(wp.word.strip()) >= 2
                and wp.word.lower() not in self.stop_words)

    def textrank(self, sentences, topK=20):
        """
        Extract keywords from sentence using TextRank algorithm.
        Parameter:
            - topK: return how many top keywords. `None` for all possible words.
            - withWeight: if True, return a list of (word, weight);
                          if False, return a list of words.
            - allowPOS: the allowed POS list eg. ['ns', 'n', 'vn', 'v'].
                        if the POS of w is not in this list, it will be filtered.
            - withFlag: if True, return a list of pair(word, weight) like posseg.cut
                        if False, return a list of words
        """
#         self.pos_filt = frozenset(allowPOS)
        g = UndirectWeightedGraph()
        cm = defaultdict(int)
#         words = tuple(self.tokenizer.cut(sentence))
        for i in range(len(sentences)):
            words=sentences[i].split(" ")
            for j, wp in enumerate(words):
                for k in range(j, j + self.span):
                    if k >= len(words):
                        break
                    cm[(wp, words[k])] += 1

        for terms, w in cm.items():
            g.addEdge(terms[0], terms[1], w)
        nodes_rank = g.rank()
        tags = sorted(nodes_rank, key=nodes_rank.__getitem__, reverse=True)

        if topK:
            return tags[:topK]
        else:
            return tags

In [9]:
import numpy as np
import math
import gensim
import random
from collections import Counter

train1=[]
train2=[]
train_1=[]
train_2=[]
trainn_1=[]
trainn_2=[]
c_set=30
min_len=6
max_len=100
min_rate=0.6
max_rate=0.8
base_count=(max_len-min_len)/(max_rate-min_rate)
mark_count=10
comlen=10
wordstop100=[]
train_tag=[]
trainn_tag=[]

for i in range(len(x_train)):
    random.shuffle(x_train[i])
    
for i in range(len(x_train)):
    print(i)
    x=""
    for k in range(comlen):
        if k==0:
            x+=x_train[i][k]
        else:
            x+=" "+x_train[i][k]
        if len(''.join(x.split(' ')))>200:
            break
    j=comlen
    while j<180:
        y=""
        for k in range(comlen):
            if k==0:
                y+=x_train[i][j+k]
            else:
                y+=" "+x_train[i][j+k]
            if len(''.join(y.split(' ')))>200:
                break
        train_1.append(x)
        train_2.append(y)
        train_tag.append(i)
        j+=1+k
    j=i+1
    while j<len(x_train) and j<i+40:
        y=""
        if len(x_train[j])<10:
            j+=1
            continue
        l=random.sample(range(len(x_train[j])),comlen)
        for k in range(len(l)):
            if k==0:
                y+=x_train[j][l[k]]
            else:
                y+=" "+x_train[j][l[k]]
            if len(''.join(y.split(' ')))>200:
                break
        trainn_1.append(x)
        trainn_2.append(y)
        trainn_tag.append((i,j))
        j+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [4]:
import numpy as np
import math
import gensim
import random
from collections import Counter

train1=[]
train2=[]

train_1=[]
train_2=[]

trainn_1=[]
trainn_2=[]
textrank=TextRank()
text_group=5
comlen=10
wordstoprank1001=[]
wordstoprank1002=[]
train_tag=[]
trainn_tag=[]

for i in range(len(x_train)):
    random.shuffle(x_train[i])
    
for i in range(len(x_train)):
    j=0
    train1.append([])
    while j<len(x_train[i]):
        y=""
        for k in range(comlen):
            if j+k<len(x_train[i]):
                if k==0:
                    y+=x_train[i][j+k]
                else:
                    y+=" "+x_train[i][j+k]
                if len(''.join(y.split(' ')))>200:
                    break
            else:
                break
        train1[-1].append(y)
        j+=1+k

i=0
while i<len(train1):
#     if len(train1[i])<20:
#         del train1[i]
#         continue
    train2.append([])
    x=math.floor(len(train1[i])/2)
    train2[-1]=train1[i][x:]
    train1[i]=train1[i][0:x]
    i+=1
for i in range(len(train1)):
    wordstoprank1001.append([])
    for j in range(math.floor(len(train1[i])/text_group)):
        wordstoprank1001[-1].append(textrank.textrank(train1[i][j*text_group:(j+1)*text_group],100))
for i in range(len(train2)):
    wordstoprank1002.append([])
    for j in range(math.floor(len(train2[i])/text_group)):
        wordstoprank1002[-1].append(textrank.textrank(train2[i][j*text_group:(j+1)*text_group],100))

In [10]:
import numpy as np
import math
import gensim
import random
from collections import Counter
# from langconv import *
# def tradition2simple(line):
#     # 将繁体转换成简体
#     line = Converter('zh-hans').convert(line)
#     return line

# EMBEDDING_FILE='/home/sysu502/Public/duxin/wiki/wiki/word2vec_wiki.model'
# mod = gensim.models.deprecated.word2vec.Word2Vec.load(EMBEDDING_FILE)
test_1=[]
test_2=[]
testn_1=[]
testn_2=[]
c_set=30
min_len=6
max_len=100
min_rate=0.6
max_rate=0.8
base_count=(max_len-min_len)/(max_rate-min_rate)
mark_count=10
comlen=10
wordstopt100=[]
test_tag=[]
testn_tag=[]

for i in range(len(x_test)):
    random.shuffle(x_test[i])
    
#记得处理停用词和标点
for i in range(len(x_test)):
    print(i)
    x=""
    for k in range(comlen):
        if k==0:
            x+=x_test[i][k]
        else:
            x+=" "+x_test[i][k]
        if len(''.join(x.split(' ')))>200:
            break
    j=comlen
    while j<180:
        y=""
        for k in range(comlen):
            if k==0:
                y+=x_test[i][j+k]
            else:
                y+=" "+x_test[i][j+k]
            if len(''.join(y.split(' ')))>200:
                break
        test_1.append(x)
        test_2.append(y)
        test_tag.append(i)
        j+=1+k
    j=i+1
    while j<len(x_test):
        y=""
        if len(x_test[j])<10:
            j+=1
            continue
        l=random.sample(range(len(x_test[j])),comlen)
        for k in range(len(l)):
            if k==0:
                y+=x_test[j][l[k]]
            else:
                y+=" "+x_test[j][l[k]]
            if len(''.join(y.split(' ')))>200:
                break
        testn_1.append(x)
        testn_2.append(y)
        testn_tag.append((i,j))
        j+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [5]:
import numpy as np
import math
import gensim
import random
from collections import Counter

test1=[]
test2=[]

test_1=[]
test_2=[]

testn_1=[]
testn_2=[]

comlen=10
wordstoptrank1001=[]
wordstoptrank1002=[]
test_tag=[]
testn_tag=[]

for i in range(len(x_test)):
    random.shuffle(x_test[i])
    
for i in range(len(x_test)):
    j=0
    test1.append([])
    while j<len(x_test[i]):
        y=""
        for k in range(comlen):
            if j+k<len(x_test[i]):
                if k==0:
                    y+=x_test[i][j+k]
                else:
                    y+=" "+x_test[i][j+k]
                if len(''.join(y.split(' ')))>200:
                    break
            else:
                break
        test1[-1].append(y)
        j+=1+k

i=0
while i<len(test1):
#     if len(train1[i])<20:
#         del train1[i]
#         continue
    test2.append([])
    x=math.floor(len(test1[i])/2)
    test2[-1]=test1[i][x:]
    test1[i]=test1[i][0:x]
    i+=1
for i in range(len(test1)):
    wordstoptrank1001.append([])
    for j in range(math.floor(len(test1[i])/text_group)):
        wordstoptrank1001[-1].append(textrank.textrank(test1[i][j*text_group:(j+1)*text_group],100))
for i in range(len(test2)):
    wordstoptrank1002.append([])
    for j in range(math.floor(len(test2[i])/text_group)):
        wordstoptrank1002[-1].append(textrank.textrank(test2[i][j*text_group:(j+1)*text_group],100))

In [7]:
import numpy as np
import math
import gensim
import random
from collections import Counter

test1=[]
test2=[]

input1 = open('/home/sysu2018/Public/duxin/content_new961_idf.txt', 'r',encoding="utf-8")
idf=eval(input1.read())
input1.close()

# test_1=[]
# test_2=[]

# testn_1=[]
# testn_2=[]

textrank=TextRank()

comlen=10
wordstopt1001=[]
wordstopt1002=[]
wordstopttf1001=[]
wordstopttf1002=[]
wordstoptrank1001=[]
wordstoptrank1002=[]
# test_tag=[]
# testn_tag=[]

for i in range(len(x_test)):
    random.shuffle(x_test[i])

for i in range(len(x_test)):
    j=0
    test1.append([])
    while j<len(x_test[i]):    
        y=""
        for k in range(comlen):
            if j+k<len(x_test[i]):
                if k==0:
                    y+=x_test[i][j+k]
                else:
                    y+=" "+x_test[i][j+k]
                if len(''.join(y.split(' ')))>200:
                    break
            else:
                break
        test1[-1].append(y)
        j+=1+k

i=0
while i<len(test1):
#     if len(test1[i])<20:
#         del test1[i]
#         continue
    test2.append([])
    x=math.floor(len(test1[i])/2)
    test2[-1]=test1[i][x:]
    test1[i]=test1[i][0:x]
    i+=1
for i in range(len(test1)):
    wordstopt1001.append(Counter())
    for j in range(len(test1[i])):
        words=test1[i][j].split(" ")
        for k in range(len(words)):
            wordstopt1001[-1][words[k]]+=1
    wordstopt1001[-1]=Counter(dict(wordstopt1001[-1].most_common(800)))
for i in range(len(test2)):
    wordstopt1002.append(Counter())
    for j in range(len(test2[i])):
        words=test2[i][j].split(" ")
        for k in range(len(words)):
            wordstopt1002[-1][words[k]]+=1
    wordstopt1002[-1]=Counter(dict(wordstopt1002[-1].most_common(800)))

for i in range(len(test1)):
    wordstopttf1001.append(Counter())
    for j in range(len(test1[i])):
        words=test1[i][j].split(" ")
        for k in range(len(words)):
            wordstopttf1001[-1][words[k]]+=1
    for key in wordstopttf1001[-1]:
        wordstopttf1001[-1][key]=wordstopttf1001[-1][key]*idf[key]
    wordstopttf1001[-1]=Counter(dict(wordstopttf1001[-1].most_common(800)))
for i in range(len(test2)):
    wordstopttf1002.append(Counter())
    for j in range(len(test2[i])):
        words=test2[i][j].split(" ")
        for k in range(len(words)):
            wordstopttf1002[-1][words[k]]+=1
    for key in wordstopttf1002[-1]:
        wordstopttf1002[-1][key]=wordstopttf1002[-1][key]*idf[key]
    wordstopttf1002[-1]=Counter(dict(wordstopttf1002[-1].most_common(800)))

for i in range(len(test1)):
    wordstoptrank1001.append(textrank.textrank(test1[i],800))
for i in range(len(test2)):
    wordstoptrank1002.append(textrank.textrank(test2[i],800))

In [10]:
print(len(train_1))
print(len(trainn_1))
print(len(test_1))
print(len(testn_1))

0
0
0
0


In [8]:
for i in range(len(wordstopttf1002)):
    if len(wordstopttf1002[i])<800:
        print(i)

In [7]:
test1=[]
test2=[]
wordstop1001=[]
wordstop1002=[]
for i in range(len(x_test)):
    print(i)
    test1.append([])
    j=0
    while j<len(x_test[i]):
        y=""
        for k in range(10):
            if j+k<len(x_test[i]):
                if k==0:
                    y+=x_test[i][j+k]
                else:
                    y+=" "+x_test[i][j+k]
                if len(''.join(y.split(' ')))>200:
                    break
            else:
                break
            
        test1[-1].append(y)
        j+=1+k

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [8]:
import math
i=0
while i<len(test1):
    if len(test1[i])<20:
        del test1[i]
        continue
    test2.append([])
    x=math.floor(len(test1[i])/2)
    test2[-1]=test1[i][x:]
    test1[i]=test1[i][0:x]
    i+=1
for i in range(len(test1)):
    wordstop1001.append(Counter())
    for j in range(len(test1[i])):
        words=test1[i][j].split(" ")
        for k in range(len(words)):
            wordstop1001[-1][words[k]]+=1
    wordstop1001[-1]=Counter(dict(wordstop1001[-1].most_common(100)))
for i in range(len(test2)):
    wordstop1002.append(Counter())
    for j in range(len(test2[i])):
        words=test2[i][j].split(" ")
        for k in range(len(words)):
            wordstop1002[-1][words[k]]+=1
    wordstop1002[-1]=Counter(dict(wordstop1002[-1].most_common(100)))

In [11]:
import torch
ac=0
# model=torch.load("model0.7304561298768334.pth")
# model=model.eval()
ra=[]
ra1=[]
ratf=[]
raidf=[]
rarank=[]

# for i in range(len(test1)):
#     wordt_list1.append([])
#     for j in range(len(test1[i])):
#         x=test1[1][j].split(" ")
#         if maxl<len(x):
#             maxl=len(x)
#         x1= [word_to_idx[j1] for j1 in x]
#         wordt_list1[-1].append()

for i in range(len(test1)):
    ra.append([])
    ra1.append([])
    ratf.append([])
    raidf.append([])
    rarank.append([])
    for j in range(len(test2)):
        test_1=[]
        test_2=[]
        for k in range(len(test1[i])):
            for k1 in range(len(test2[j])):
                test_1.append(test1[i][k])
                test_2.append(test2[j][k1])
        test_data=[]
        maxl=0
        for i1 in range(len(test_1)):
            x=test_1[i1].split(" ")
            if maxl<len(x):
                maxl=len(x)
            x1= [word_to_idx[j1] for j1 in x]
            x=test_2[i1].split(" ")
            if maxl<len(x):
                maxl=len(x)
            x2= [word_to_idx[j1] for j1 in x]
            test_data.append(([x1,x2],1))

        args['max_seq_len']=maxl
        inputt1_len=[]
        inputt2_len=[]
        wordt_list1=[]
        wordt_list2=[]

        for i1 in range(len(test_data)):
            word,tag= test_data[i1]
            for j1 in range(len(word)):
                if j1==0:
                    inputt1_len.append(len(word[j1]))
                else:
                    inputt2_len.append(len(word[j1]))
                while 1:
                    if len(word[j1])<args['max_seq_len']:
                        word[j1].append(0)
                    else:
                        break
            wordt_list1.append(word[0])
            wordt_list2.append(word[1])

        inputts1=torch.LongTensor(wordt_list1)
        inputts2=torch.LongTensor(wordt_list2)

        if torch.cuda.is_available():
            inputts1 = inputts1.cuda()
            inputts2 = inputts2.cuda()

        count1=0
        count2=0
        count3=0
        count4=0
        count5=0
        for i1 in range(len(test_data)):
            j1=len(inputts1[i1])-1
            while 1:
                if inputts1[i1,j1]==0:
                    j1-=1
                else:
                    break

            j2=len(inputts2[i1])-1
            while 1:
                if inputts2[i1,j2]==0:
                    j2-=1
                else:
                    break

            test_output = model(inputts1[i1,:j1+1].unsqueeze(0), inputts2[i1,:j2+1].unsqueeze(0))
            
            pred_y = torch.max(test_output.cpu(), 1)[1].data.numpy().squeeze()
            if pred_y==1:
                count2+=1
            for i2 in range(len(test_output.cpu())):
                count1+=F.softmax(test_output.cpu()[i2])[1].data.numpy().squeeze()
            optimizer.zero_grad()
        ra[-1].append(count1/len(test_data))
        ra1[-1].append(count2/len(test_data))
        
        test_output = model(test1_att[i].unsqueeze(0),test2_att[j].unsqueeze(0))
        for i2 in range(len(test_output.cpu())):
            count3+=F.softmax(test_output.cpu()[i2])[1].data.numpy().squeeze()
        
        test_output = model(test3_att[i].unsqueeze(0),test4_att[j].unsqueeze(0))
        for i2 in range(len(test_output.cpu())):
            count4+=F.softmax(test_output.cpu()[i2])[1].data.numpy().squeeze()
            
        test_output = model(test5_att[i].unsqueeze(0),test6_att[j].unsqueeze(0))
        for i2 in range(len(test_output.cpu())):
            count5+=F.softmax(test_output.cpu()[i2])[1].data.numpy().squeeze()
            
        ratf[-1].append(count3)
        raidf[-1].append(count4)
        rarank[-1].append(count5)
        
    print(i)
    print(max(ra[-1]),max(ra1[-1]),max(ratf[-1]),max(raidf[-1]),max(rarank[-1]))
    print(ra[-1][i],ra1[-1][i],ratf[-1][i],raidf[-1][i],rarank[-1][i])

ra=np.matrix(ra)
rat=ra.T
ac=0
maac=0
for i in range(len(ra)):
    r1=np.argmax(ra[i])
    r2=np.argmax(rat[i])
    if r1==r2:
        maac+=1
    if r1==r2 and r1==i:
        ac+=1
recall=ac/len(ra)
precise=ac/maac
print("recall:",recall)
print("precise",precise)
print("f1:",2*recall*precise/(recall+precise))
# recall: 0.48936170212765956
# precise 0.9583333333333334
# f1: 0.6478873239436619

/home/sysu2018/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/sysu2018/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:107: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/sysu2018/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:111: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/sysu2018/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:115: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0
0.7979478872504694 0.9260204081632653 0.8374118804931641 0.8468661308288574 0.8765908479690552
0.7979478872504694 0.9260204081632653 0.8263314962387085 0.8200780749320984 0.8439550995826721
1
0.680796604577478 0.8321678321678322 0.7534127831459045 0.6445483565330505 0.7802523970603943
0.5447516265000112 0.6153846153846154 0.6578428149223328 0.5015740394592285 0.7349832057952881
2
0.8522536060362935 0.9204204204204204 0.8029087781906128 0.7707353234291077 0.8022612929344177
0.8522536060362935 0.9204204204204204 0.7379116415977478 0.7707353234291077 0.7227002382278442
3
0.9105122293949127 0.9952 0.8031391501426697 0.8136804699897766 0.8482223153114319
0.9105122293949127 0.9952 0.7830571532249451 0.7749911546707153 0.7628447413444519
4
0.8581468451860522 0.9669421487603306 0.8535425662994385 0.8759013414382935 0.8839542865753174
0.8581468451860522 0.9669421487603306 0.8460573554039001 0.8419272899627686 0.8127007484436035
5
0.6405529680232016 0.7646153846153846 0.78849196434021 0.778414

44
0.5933344975162885 0.6929824561403509 0.6817144751548767 0.7157018184661865 0.705075204372406
0.50983903862727 0.5705128205128205 0.5069482326507568 0.5671470165252686 0.5417092442512512
45
0.6247348983467854 0.6974789915966386 0.740994930267334 0.7663410305976868 0.690714418888092
0.5861049646213408 0.6568627450980392 0.7150290012359619 0.6987098455429077 0.6131554841995239
46
0.8324261544246434 0.9506944444444444 0.9167168140411377 0.9129583239555359 0.9155132174491882
0.81524210236058 0.9408536585365853 0.8598092198371887 0.8912459015846252 0.8957359194755554
47
0.740212342677772 0.8742138364779874 0.8197256922721863 0.8660736083984375 0.8831551671028137
0.740212342677772 0.8742138364779874 0.8197256922721863 0.8660736083984375 0.8831551671028137
48
0.5255316419309749 0.567193675889328 0.7492517828941345 0.7180669903755188 0.742656409740448
0.4945468542194276 0.5606060606060606 0.6700057983398438 0.6205667853355408 0.6613075137138367
49
0.90681973320211 0.9574759945130316 0.89369

88
0.8459415464041133 0.9125 0.8452849388122559 0.8847704529762268 0.8028877973556519
0.8459415464041133 0.9125 0.7423678040504456 0.8459561467170715 0.7554154992103577
89
0.7992844010288475 0.9361344537815126 0.8292204737663269 0.8658027648925781 0.7807818055152893
0.7992844010288475 0.9361344537815126 0.8292204737663269 0.8658027648925781 0.7807818055152893
90
0.6910521196939888 0.7869822485207101 0.8465554714202881 0.8984063863754272 0.7206754088401794
0.6910521196939888 0.7869822485207101 0.8465554714202881 0.8132420778274536 0.7206754088401794
91
0.6911329564954034 0.8073089700996677 0.7928295731544495 0.7387921214103699 0.808596670627594
0.5956097479092375 0.673469387755102 0.7521420121192932 0.7387921214103699 0.7695782780647278
92
0.8869328280532922 0.9594202898550724 0.8244200944900513 0.8290942907333374 0.8255105018615723
0.8869328280532922 0.9594202898550724 0.8085594773292542 0.8247048854827881 0.7873662710189819
93
0.8723764137390219 0.9748148148148148 0.8753647208213806 0

In [8]:
torch.save(model, 'LLG_DecAtt2.pth')

/home/sysu502/anaconda3/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type DecAtt. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [23]:
f=open("/home/sysu502/Public/duxin/LLG1_rarank.txt","w",encoding="utf-8")
f.write(str(rarank))
f.close()

In [42]:
ra11=[]
ra12=[]
ra13=[]
ra21=[]
ra22=[]
ra23=[]
alpha=1
for i in range(len(ra)):
    ra11.append([])
    ra12.append([])
    ra13.append([])
    ra21.append([])
    ra22.append([])
    ra23.append([])
    for j in range(len(ra[i])):
        ra11[-1].append(ra[i][j]*alpha+ratf[i][j]*(1-alpha))
        ra12[-1].append(ra[i][j]*alpha+raidf[i][j]*(1-alpha))
        ra13[-1].append(ra[i][j]*alpha+rarank[i][j]*(1-alpha))
        ra21[-1].append(ra1[i][j]*alpha+ratf[i][j]*(1-alpha))
        ra22[-1].append(ra1[i][j]*alpha+raidf[i][j]*(1-alpha))
        ra23[-1].append(ra1[i][j]*alpha+rarank[i][j]*(1-alpha))

In [47]:
count=100
ra2=np.matrix(ra11[0:count][0:count])
ra3=np.matrix(ra12[0:count][0:count])
ra4=np.matrix(ra13[0:count][0:count])
ra5=np.matrix(ra21[0:count][0:count])
ra6=np.matrix(ra22[0:count][0:count])
ra7=np.matrix(ra23[0:count][0:count])
ac1=0
maac1=0
ac2=0
maac2=0
ac3=0
maac3=0
ac4=0
maac4=0
ac5=0
maac5=0
ac6=0
maac6=0
for i in range(len(ra2)):
    r1=np.argmax(ra2[i])
    r2=np.argmax(ra2[:,i])
    r3=np.argmax(ra3[i])
    r4=np.argmax(ra3[:,i])
    r5=np.argmax(ra4[i])
    r6=np.argmax(ra4[:,i])
    r7=np.argmax(ra5[i])
    r8=np.argmax(ra5[:,i])
    r9=np.argmax(ra6[i])
    r10=np.argmax(ra6[:,i])
    r11=np.argmax(ra7[i])
    r12=np.argmax(ra7[:,i])
    if r1==r2:
        maac1+=1
    if r1==r2 and r1==i:
        ac1+=1
    if r3==r4:
        maac2+=1
    if r3==r4 and r3==i:
        ac2+=1
    if r5==r6:
        maac3+=1
    if r5==r6 and r5==i:
        ac3+=1
    if r7==r8:
        maac4+=1
    if r7==r8 and r7==i:
        ac4+=1
    if r9==r10:
        maac5+=1
    if r9==r10 and r9==i:
        ac5+=1
    if r11==r12:
        maac6+=1
    if r11==r12 and r11==i:
        ac6+=1
recall1=ac1/len(ra2)
precise1=ac1/maac1
recall2=ac2/len(ra2)
precise2=ac2/maac2
recall3=ac3/len(ra2)
precise3=ac3/maac3
recall4=ac4/len(ra2)
precise4=ac4/maac4
recall5=ac5/len(ra2)
precise5=ac5/maac5
recall6=ac6/len(ra2)
precise6=ac6/maac6
print(recall1,recall2,recall3,recall4,recall5,recall6)
print(precise1,precise2,precise3,precise4,precise5,precise6)
print(2*recall1*precise1/(recall1+precise1),2*recall2*precise2/(recall2+precise2),2*recall3*precise3/(recall3+precise3),2*recall4*precise4/(recall4+precise4),2*recall5*precise5/(recall5+precise5),2*recall6*precise6/(recall6+precise6))

0.56 0.56 0.56 0.52 0.52 0.52
0.9491525423728814 0.9491525423728814 0.9491525423728814 0.9285714285714286 0.9285714285714286 0.9285714285714286
0.7044025157232704 0.7044025157232704 0.7044025157232704 0.6666666666666666 0.6666666666666666 0.6666666666666666


In [12]:
import torch
torch.LongTensor([[1,2,3],
                 [4,5,6]])

tensor([[1, 2, 3],
        [4, 5, 6]])